In [33]:
import glob
from os import path
import torch
import torchvision.transforms as transforms
import cv2
import numpy as np
from utils_dataset import get_images_labels
from utils_image import imread_rgb
from utils_params import match_label_fou_clean2, match_label_video

In [34]:
dataset_path = 'data/pre_proc/CLICIDE_video_224sq'
match_labels = match_label_video
image_size = None

In [35]:
dataset_full = get_images_labels(dataset_path, match_labels)

In [36]:
mean = [0., 0., 0.]
std = [0., 0., 0.]
size = len(dataset_full)
if image_size is not None:
    T = torch.Tensor(size, *(image_size))
    for i, (im, _) in enumerate(dataset_full):
        T[i] = transforms.ToTensor()(imread_rgb(im))
    for i in range(3):
        mean[i] = T[:, i, :, :].mean()
        std[i] = T[:, i, :, :].std()
else:
    # cannot take mean/std of whole dataset tensor.
    # need to compute mean of all pixels and std afterwards, pixel by pixel
    dataset_open = []
    for im, _ in dataset_full:
        im_o = imread_rgb(im) / 255.  # cv2 images are 0-255, torch tensors are 0-1
        im_size = im_o.shape[0] * im_o.shape[1]
        dataset_open.append((im_o, im_size))
        for i in range(3):
            mean[i] += np.sum(im_o[:, :, i]) / (im_size * size)
    for im_o, im_size in dataset_open:
        for i in range(3):
            std[i] += np.sum(np.square(im_o[:, :, i] - mean[i])) / (im_size * size)
    for i in range(3):
        std[i] = np.sqrt(std[i])
print(mean)
print(std)

[0.42547604114835502, 0.42643734822029833, 0.41766329596168206]
[0.2215131390640957, 0.21265148289169833, 0.20964298884822521]
